Download torch and torchvision

In [ ]:
# pip install torch

In [ ]:
# pip install torchvision

Initialization

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler


import torchvision.datasets as dset
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.transforms import v2

import numpy as np

USE_GPU = True
dtype = torch.float32 
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    device = torch.device('cpu')
    print("No GPU available. Training will run on CPU.")


# Constant to control how frequently we print train loss.
print_every = 100

GPU: NVIDIA GeForce RTX 4080 is available.


If the output is "using device: cpu": download CUDA toolkit (https://developer.nvidia.com/cuda-downloads)
https://pytorch.org/get-started/locally/

In [ ]:
# data augmentation
transform = v2.Compose([
                v2.ToImage(), 
                v2.ToDtype(torch.float32, scale=True),
                v2.RandomVerticalFlip(),
                v2.ColorJitter(brightness=(0.5, 1.5), contrast=(0.5, 1.5), saturation=(0.5, 1.5), hue=(-0.3, 0.3)),
                v2.RandomChannelPermutation(),
                v2.RandomErasing(p=0.5, scale=(0.05, 0.3), ratio=(1, 1), value='random'),
                v2.RandomRotation(degrees=30)])

# Split dataset -> train, val, test
dataset = dset.ImageFolder('ClassificationDataset_label', transform=transform)
total_len = len(dataset)

NUM_TRAIN = int(0.8 * total_len)
NUM_VAL = int(0.1 * total_len)

indices = list(range(total_len))
train_indices = indices[:NUM_TRAIN]
val_indices = indices[NUM_TRAIN:NUM_TRAIN+NUM_VAL]
test_indices = indices[NUM_TRAIN+NUM_VAL:]

loader_train = DataLoader(dataset, batch_size=128, sampler=SubsetRandomSampler(train_indices))
loader_val = DataLoader(dataset, batch_size=64,
                        sampler=SubsetRandomSampler(val_indices))
loader_test = DataLoader(dataset, batch_size=64,
                         sampler=SubsetRandomSampler(test_indices))

Check If load correctly

In [15]:
loader_train

In [26]:
for images, labels in loader_val:
    print("Batch loaded!")
    print("Images shape:", images.shape)
    print("Labels:", labels)
    break


Batch loaded!
Images shape: torch.Size([25, 3, 320, 320])
Labels: tensor([1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1,
        0])


Model setup

In [28]:
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [ ]:
def check_accuracy_final(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        return acc

def train(model, optimizer, epochs=10):
    x1 = list(range(len(loader_train) * epochs))
    x2 = []
    y1 = []
    y2 = []
    model = model.to(device=device)
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            y1.append(loss.item())
            if t % print_every == 0:
                x2.append(e * len(loader_train) + t)
                y2.append(check_accuracy_final(loader_val, model))
    return (x1, y1, x2, y2)

def plotpic(data):
  x1, y1, x2, y2 = data
  plt.figure()
  plt.plot(x1, y1, marker='.')
  plt.xlabel('Iteratoin')
  plt.ylabel('Loss')
  plt.title('Loss')
  plt.grid(True)

  plt.figure()
  plt.plot(x2, y2, marker='o')
  plt.xlabel('Iteratoin')
  plt.ylabel('Val Acc')
  plt.title('Val Acc')
  plt.grid(True)
  plt.show()

In [34]:
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.Conv2d(32, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Dropout(0.4),
    
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Dropout(0.4),

    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Dropout(0.4),

    nn.Flatten(),
    nn.Linear(128 * 40 * 40, 512),
    nn.Linear(512, 256),
)

In [35]:
sgdMNb = optim.SGD(model.parameters(), lr=1e-3,
                     momentum=0.9, nesterov=True)

Train

In [ ]:
data = train(model, sgdMNb, epochs=1)

In [ ]:
plotpic(data1)